# Capítulo 5

O objetivo do início do capítulo é mostrar como usar a biblioteca `dplyr` pode manipular os dados para que eles atendam as necessidades de um plot, e como usar as pipelines(`%>%`) para realizar multiplas operações com os bancos de dados e as funções dessa biblioteca, com uma sintaxe bem dinâmica.

Antes de tudo, é importante importar as bibliotecas que serão usadas:

In [4]:
import altair as alt
import pandas as pd
from pysocviz.loader import load_dataset
gss_sm = load_dataset("gss_sm")
elections = load_dataset("elections_historic")
organdata = pd.read_csv("organdata.csv")
organdata["year"] = pd.to_datetime(organdata["year"])

Como é de se esperar, a maniplação da base de dados no python é feita de maneira bem diferente do R. Aqui, a biblioteca responsável por manipular os dados é a mesma responsável por organizá-los em tabela, a `pandas`. Na construção dos gráficos com aparições relativas de religião do capítulo anterior, já foram usadas algumas funções simples dessa biblioteca para calcular tabelas com a porcentagem das aparições das religiões ou regiões. Refazendo elas:



In [5]:
#Cria uma tabela com a contagem das aparições de cada combinação de região e religião
gss_sm_mod2 = gss_sm[["religion", "bigregion"]].value_counts(dropna = False).to_frame().reset_index()
#Salva o total de aparições de cada religião
totais = gss_sm["religion"].value_counts(dropna = False).to_dict()
#Cria uma coluna freq, que exibe a porcentagem relativa de cada religião.
gss_sm_mod2["freq"] = gss_sm_mod2.apply(lambda reg: reg["count"]/totais[reg["religion"]], axis= 1)
gss_sm_mod2

,religion,bigregion,count,freq
0,Protestant,South,650,0.474106
1,Protestant,Midwest,325,0.237053
2,Protestant,West,238,0.173596
3,NaN,West,181,0.284144
4,NaN,South,181,0.284144
5,Catholic,Midwest,172,0.265023
6,Catholic,Northeast,162,0.249615
7,NaN,Midwest,162,0.254317
8,Catholic,South,160,0.246533
9,Protestant,Northeast,158,0.115244


In [6]:
#Obtem uma tabela com a contagem das aparições de cada combinação entre região e religião
gss_sm_mod3 = gss_sm[["religion", "bigregion"]].value_counts(dropna = False).to_frame().reset_index()
#Desta vez, salva a contagem das aparições de cada região, e não religião.
totais = gss_sm["bigregion"].value_counts(dropna = False).to_dict()
#Calcula as porcentagens de aparição de cada religião, por cada região.
gss_sm_mod3["freq"] = gss_sm_mod3.apply(lambda reg: reg["count"]/totais[reg["bigregion"]], axis= 1)
gss_sm_mod3

,religion,bigregion,count,freq
0,Protestant,South,650,0.617871
1,Protestant,Midwest,325,0.467626
2,Protestant,West,238,0.376582
3,NaN,West,181,0.286392
4,NaN,South,181,0.172053
5,Catholic,Midwest,172,0.247482
6,Catholic,Northeast,162,0.331967
7,NaN,Midwest,162,0.233094
8,Catholic,South,160,0.152091
9,Protestant,Northeast,158,0.323770


Operações com objetivo semelhante são feitas logo no começo do capítulo, para plotar um gráfico que exibe as religiões em porcentagens por região. Esse gráfico já pode ser reproduzido, justamente por essa tabela já ter sido calculada:

In [7]:
alt.Chart(gss_sm_mod3).mark_bar().encode(
    x = "bigregion",
    xOffset = "religion",
    y = "freq",
    color = "religion",
)

alt.Chart(...)

O livro em seguida propõe que os gráficos sejam refletidos e explica que trocar os eixos X e Y não é o suficiente para isso funcionar. No caso do altair, isso é possível:

In [8]:
alt.Chart(gss_sm_mod3).mark_bar().encode(
    y = "bigregion",
    yOffset = "religion",
    x = "freq",
    color = "religion",
)

alt.Chart(...)

Facetando:

In [9]:
alt.Chart(gss_sm_mod3).mark_bar().encode(
    y = "religion",
    column = "bigregion",
    x = "freq",
    color = "religion",
).properties(width = 150, height = 200)

alt.Chart(...)

O livro prossegue dando exemplos de como usar pipes para obter um exercto da base de dados. Para obter algo nas mesmas dimensões obtidas, é preciso acessar o atributo `iloc`:

In [10]:
organdata.iloc[0:10, 0:6]

,country,year,donors,pop,pop_dens,gdp
0,Australia,NaT,NaN,17065.0,0.220443,16774.0
1,Australia,1991-01-01,12.09,17284.0,0.223272,17171.0
2,Australia,1992-01-01,12.35,17495.0,0.225998,17914.0
3,Australia,1993-01-01,12.51,17667.0,0.228220,18883.0
4,Australia,1994-01-01,10.25,17855.0,0.230648,19849.0
5,Australia,1995-01-01,10.18,18072.0,0.233452,21079.0
6,Australia,1996-01-01,10.59,18311.0,0.236539,21923.0
7,Australia,1997-01-01,10.26,18518.0,0.239213,22961.0
8,Australia,1998-01-01,10.48,18711.0,0.241706,24148.0
9,Australia,1999-01-01,8.67,18926.0,0.244483,25445.0


O primeiro gráfico real feito com essa base de dados é um facetado com os países e a quantidade de doadores ao longo dos anos:

In [11]:
alt.Chart(organdata).mark_line().encode(
    x = "year",
    y = "donors",
    facet = alt.Facet("country", columns = 5)
).properties(height = 160, width = 160)

alt.Chart(...)

Uma pequena modificação que poderia contribuir para deixar esse conjunto de gráficos menos poluído é sumir com as gridlines dos eixos:

In [12]:
alt.Chart(organdata).mark_line().encode(
    x = "year",
    y = "donors",
    facet = alt.Facet("country", columns = 5)
).properties(height = 160, width = 160).configure_axis(grid = False)

alt.Chart(...)

Quando o objetivo não é observar a variação do número de doadores ao longo dos anos, mas sim apenas observar a distribuição desses números por país, é possível usar gráficos de boxplots. Os boxplots no altair são feitos usando a função `mark_boxplot()`, como é de se esperar. Caso o desejo fosse fazer um único box plot para toda a base de dados, o procedimento seria passar apenas o eixo y para o método `encode`:

In [13]:
alt.Chart(organdata).mark_boxplot().encode(
    y = "donors"
)

alt.Chart(...)

Como queremos os boxplots por países, passamos `"country"` como variável do eixo x:

In [14]:
alt.Chart(organdata).mark_boxplot().encode(
    x = "country",
    y = "donors",
)

alt.Chart(...)

Vale ressaltar que por padrão os boxplots do altair são interativos, e exibem as medidas estatísticas (quartis e mediana) de cada caixa quando se passa o mouse por cima delas. Para inverter os eixos, não é preciso nenhuma função especial, apenas trocar as variáveis na especificação dos canais já funciona:

In [15]:
alt.Chart(organdata).mark_boxplot().encode(
    y = "country",
    x = "donors",
)

alt.Chart(...)

Para mudar a ordenação dos países, é preciso específicar ou uma lista com a ordem a ser usada (técnica usada em gráficos do capítulo passado). 

Em teoria, a biblioteca deveria aceitar um objeto `SortField` (que foi usado mas não explicado em um gráfico anterior), o qual diz qual critério de ordenação deve ser usado. Esse recurso, porém, não funciona com boxplots devido a [problemas internos da biblioteca](https://github.com/altair-viz/altair/issues/2322).

Logo, para ordenar os boxplots, é necessário calcular a lista com a ordem antes de plotar o gráfico. Para isso, primeiro se escolhe as colunas que vão ser usadas na operação (acessando elas com uma lista), em seguida a função `groupby` agrupa pelos países e por fim a função `mean` retorna a média dos números de doadores de cada país:

In [16]:
organdata_medias = organdata[["country", "donors"]].groupby("country").mean()
organdata_medias

,donors
country,
Australia,10.635000
Austria,23.525000
Belgium,21.900000
Canada,13.966667
Denmark,13.091667
Finland,18.441667
France,16.758333
Germany,13.041667
Ireland,19.791667


Ordenando pela média e transformando em lista:

In [17]:
organdata_ordem = list(organdata_medias.sort_values(by = "donors", ascending = False).index)
organdata_ordem

['Spain',
 'Austria',
 'Belgium',
 'United States',
 'Ireland',
 'Finland',
 'France',
 'Norway',
 'Switzerland',
 'Canada',
 'Netherlands',
 'United Kingdom',
 'Sweden',
 'Denmark',
 'Germany',
 'Italy',
 'Australia']

Essa lista é especificada como regra da ordenação para o eixo Y:

In [18]:
alt.Chart(organdata).mark_boxplot().encode(
    x = "donors",
    y = alt.Y("country").sort(organdata_ordem),
).properties(width = 500)

alt.Chart(...)

O procedimento para adicionar cores já é um velho conhecido:

In [19]:
alt.Chart(organdata).mark_boxplot().encode(
    x = "donors",
    y = alt.Y("country").sort(organdata_ordem),
    color = "world"
).properties(width = 500)

alt.Chart(...)

Já que são poucas as observações de doadores de orgãos por país, é válido não usar boxplots, mas sim ver os pontos diretamente no gráfico. Isso é facilmente alcançado quando se troca o método `mark_boxplot` pelo método `mark_circle` (outro método que já foi muito usado em exemplos passados):

In [20]:
alt.Chart(organdata).mark_circle().encode(
    x = "donors",
    y = alt.Y("country").sort(organdata_ordem),
    color = "world"
).properties(width = 500)

alt.Chart(...)

Refazer esse gráfico com ruído não é tão simples quanto trocar de geom no ggplot. Para alcançar esse objetivo, é preciso usar o canal `yOffset`, com uma variável `jitter`, que por sua vez é calculada usando uma fórmula que envolve aleatoriedade. por ser um cálculo simples, é possível fazer diretamente na criação do gráfico, usando o método `transform_calculate` (A fórmula foi retirada [desse exemplo](https://altair-viz.github.io/gallery/strip_plot_jitter.html)).

In [21]:
alt.Chart(organdata).mark_circle().transform_calculate(
    jitter = "sqrt(-2*log(random()))*cos(2*PI*random())"
).encode(
    x = "donors",
    y = alt.Y("country").sort(organdata_ordem),
    yOffset = "jitter:Q",
    color = "world"
).properties(width = 500)

alt.Chart(...)

(O método `transform_calculate` cria uma coluna nova para cada linha do banco de dados contendo o resultado da fórmula matemática).

Uma forma inteligente de visualizar a distribuição sem ficar limitado as caixas dos boxplots, que não indicam a distribuição interna das observações, é usar os pontos com as distribuições. É possível sobrepor os pontos nos boxplots deixando eles transparentes:

In [22]:
caixas = alt.Chart(
    organdata,
    title = alt.TitleParams(
        "Doadores de órgãos, por país",
        subtitle = "Cada observação é a quantidade de doadores de órgãos que o país teve em um ano.")
).mark_boxplot(opacity = 0.3).encode(
    alt.X("donors").title("Doadores"),
    alt.Y("country").sort(organdata_ordem).title("Países"),
    alt.Color("world").scale(scheme = "turbo").title("Alinhamento político")
).properties(width = 600)
pontos = caixas.mark_circle().transform_calculate(
    jitter = "sqrt(-2*log(random()))*cos(2*PI*random())"
).encode(
    yOffset = "jitter:Q"
)
(caixas + pontos).configure_legend(
    strokeColor = "black",
    orient = "left",
    padding = 5,
    cornerRadius = 9,
    titleAlign = "center",
    titleAnchor = "middle",
)

alt.LayerChart(...)

As próximas explicações do livro se referem a como alterar a base de dados para obter as medidas estatísticas dos valores númericos , agrupadas por país. Reproduzir isso no pandas é simples: primeiro se obtem uma lista de colunas númericas, usando a função `select_dtypes`, e depois se faz o mesmo procedimento feito anteriormente para obter a lista das médias de doadores por país, mas dessa vez com mais colunas.

In [23]:
colunas_numericas = list(organdata.select_dtypes([int, float]).columns)
colunas_numericas

['donors',
 'pop',
 'pop_dens',
 'gdp',
 'gdp_lag',
 'health',
 'health_lag',
 'pubhealth',
 'roads',
 'cerebvas',
 'assault',
 'external',
 'txp_pop']

In [24]:
organdata_medidas = organdata[["country", *colunas_numericas]].groupby("country").mean()
organdata_medidas

,donors,pop,pop_dens,gdp,gdp_lag,health,health_lag,pubhealth,roads,cerebvas,assault,external,txp_pop
country,,,,,,,,,,,,,
Australia,10.635000,18317.923077,0.236628,22178.538462,21779.428571,1957.500000,1848.214286,5.676923,104.875728,557.692308,16.769231,393.000000,0.875120
Austria,23.525000,7927.307692,9.453026,23875.846154,23415.071429,1875.357143,1803.142857,5.492308,149.865413,768.846154,10.923077,506.846154,0.630843
Belgium,21.900000,10153.307692,30.674646,22499.615385,22095.928571,1958.357143,1862.428571,6.188889,154.695038,593.846154,14.307692,541.615385,0.788005
Canada,13.966667,29607.923077,0.296952,23711.076923,23353.071429,2271.928571,2163.428571,6.676923,109.260105,422.384615,16.769231,410.615385,1.048595
Denmark,13.091667,5257.153846,12.200403,23722.307692,23275.000000,2054.071429,1973.428571,6.984615,101.636346,640.692308,12.230769,532.384615,0.761033
Finland,18.441667,5111.846154,1.511710,21018.923077,20763.000000,1615.285714,1559.785714,5.861538,93.574468,771.384615,27.461538,721.923077,0.586970
France,16.758333,58055.692308,10.526871,22602.846154,22210.714286,2159.642857,2066.428571,7.076923,156.153270,432.692308,8.923077,602.692308,0.706359
Germany,13.041667,80254.846154,22.478460,22163.230769,21938.357143,2348.750000,2256.250000,8.142308,112.788734,706.769231,9.538462,391.307692,0.550848
Ireland,19.791667,3673.615385,5.227857,20824.384615,20153.642857,1479.928571,1340.785714,4.876923,117.774245,704.692308,8.538462,394.000000,0.817583


Esse processo foi feito apenas para ilustrar que é possível, já que no fim acaba sendo desnecessário, pois é possível plotar apenas os valores de média das variáveis facilmente:

In [25]:
alt.Chart(organdata).mark_circle(size = 100).encode(
    x = alt.X("mean(donors)").scale(domain = (10, 30)),
    y = alt.Y("country").sort(organdata_ordem),
    color = "consent_law"
).properties(width = 400)

alt.Chart(...)

Facetar esse gráfico em duas linhas, a essa altura, também é trivial:

In [26]:
alt.Chart(organdata).mark_circle(size = 100).encode(
    x = alt.X("mean(donors)").scale(domain = (10, 30)),
    y = alt.Y("country").sort(organdata_ordem)
).properties(width = 400).facet(
    row = "consent_law"
).resolve_scale(
    y = "independent"
)

alt.FacetChart(...)

O último método, `resolve_scale`, serve para remover as categorias sem valor de cada gráfico, e sem ela o resultado estaria cheio de lacunas, já que ambos os gráficos teriam o eixo Y com os mesmos elementos:

In [27]:
alt.Chart(organdata).mark_circle(size = 100).encode(
    x = alt.X("mean(donors)").scale(domain = (10, 30)),
    y = alt.Y("country").sort(organdata_ordem)
).properties(width = 400).facet(
    row = "consent_law"
)

alt.FacetChart(...)

Esse tipo de gráfico listando as médias, chamado de gráfico de Cleveland, pode ser apresentado com linhas representando o desvio padrão junto do ponto que representa a média. Por sorte, a biblioteca altair tem uma função do tipo `mark` para fazer essas linhas, que retira a necessidade de calcular préviamente o desvio padrão. Para usar, é necessário especificar que o desvio padrão será usado, por meio do parâmetro `extent`:

In [28]:
linhas_erro = alt.Chart(organdata).mark_errorbar(extent = "stdev").encode(
    x = alt.X("donors").scale(domain = (5, 35)),
    y = alt.Y("country").sort(organdata_ordem)
)
linhas_erro

alt.Chart(...)

Como é possível observar, apenas as linhas são marcadas. Para marcar as médias também, é preciso sobrepor outro gráfico com essa informação:

In [29]:
alt.Chart(organdata).mark_circle(size = 100, color = "#000000").encode(
    x = alt.X("mean(donors)").scale(domain = (5, 35)),
    y = alt.Y("country").sort(organdata_ordem)
) + linhas_erro

alt.LayerChart(...)

O próximo gráfico a ser representado é um dot plot, com a média de doadores por ano em um eixo e a média de mortes em estradas no outro eixo. Para marcar apeanas pontos no gráfico, a função `marck_circle` é suficiente:

In [30]:
pontos_organ = alt.Chart(organdata).mark_circle(color = "#000000").encode(
    alt.X("mean(roads)").scale(domain = (65, 165)),
    alt.Y("mean(donors)").scale(domain = (10, 30)),
    detail = "country"
)
pontos_organ

alt.Chart(...)

Para marcar textos explicando o que cada ponto representa, é preciso usar a função `mark_text` e especificar o canal `text` na função `encode`:

In [31]:
alt.Chart(organdata).mark_text().encode(
    alt.X("mean(roads)").scale(domain = (65, 165)),
    alt.Y("mean(donors)").scale(domain = (10, 30)),
    detail = "country",
    text = "country"
) + pontos_organ

alt.LayerChart(...)

Como é possível perceber, o texto por padrão é centralizado no mesmo lugar onde se encontra o ponto. Para personalizar isso, é possível passar diversos atributos de modificação da posição do texto para o método `mark_text`, como `dx`, `dy`, `align` e `angle`:

In [32]:
alt.Chart(organdata).mark_text(
    dx = 3,
    dy = 5,
    angle = 310,
    align = "left"
).encode(
    alt.X("mean(roads)").scale(domain = (65, 165)),
    alt.Y("mean(donors)").scale(domain = (10, 30)),
    detail = "country",
    text = "country"
) + pontos_organ

alt.LayerChart(...)

Infelizmente, a biblioteca altair não fornece um meio de dispor os textos automaticamente sem eles sobreporem. Tal ferramenta já foi até programada, mas não foi publicada em nenhuma versão estável da biblioteca ([esse issue](https://github.com/altair-viz/altair/issues/1731) explica a situação da ferramenta). Logo, por mais que seja possível plotar texto e pontos, não é possível deixar os textos sem se sobreporem:

In [33]:
election_pontos = alt.Chart(elections).mark_circle(
    color = "#000000"
).encode(
    alt.X("popular_pct").scale(domain = (0.29, 0.65)),
    alt.Y("ec_pct").scale(domain = (0.29, 1.01))
)
election_texto = election_pontos.mark_text(
    color = "#000000",
    dx = 3,
    dy = -3,
    angle = 10,
    align = "left"
).encode(
    text = "winner:N"
)
(election_pontos + election_texto).properties(width = 800, height = 600)#.transform_calculate(
#    label = "datum.winner + \" - \" + datum.year"
#).interactive() + bolas)

alt.LayerChart(...)

Para modificar o texto exibido (e usar mais de uma variável em sua composição) é necessário usar o método `transform_calculate`, somando as strings das variáveis desejadas:

In [34]:
election_texto = election_pontos.transform_calculate(
    label = "datum.winner + \" - \" + datum.year"
).mark_text(
    color = "#000000",
    dx = 3,
    dy = -3,
    angle = 10,
    align = "left"
).encode(
    text = "label:N"
)
(election_pontos + election_texto).properties(width = 800, height = 600)

alt.LayerChart(...)

Para compensar a impossibilidade de separar os textos, é possível usar o método `interactive` para permitir que o gráfico receba zoom. Por mais que isso contorne a dificuldade de leitura em navegadores, não é útil quando se deseja usar apenas uma imagem do gráfico.

In [35]:
election_pontos = alt.Chart(
    elections,
    title = alt.TitleParams( 
        "Eleições presidenciais dos EUA",
         subtitle = "Comparação entre porcentagem de votos gerais e votos de colégios eleitorais")
).mark_circle(
    color = "#000000"
).encode(
    alt.X("popular_pct").scale(domain = (0.29, 0.67)).axis(format = "%").title("Votos populares"),
    alt.Y("ec_pct").scale(domain = (0.29, 1.01)).axis(format = "%").title("Votos de colégios eleitorais")
)
(election_pontos + election_texto).interactive().properties(
    width = 800, height = 600
)

alt.LayerChart(...)

Filtrar quais pontos serão nomeados é uma operção mais complicada, mas possível. Primeiro, é preciso adicionar explicitamente (usando o método `transform_aggregate`) à base de dados os campos de média, pois não é possivel usar `mean()` no texto de filtro. Após isso, o método `transform_filter` deixa apenas as linhas que se adequam a alguma das condições definidas. O resultado é o desejado:

In [36]:
out_pontos = alt.Chart(organdata).mark_circle(color = "#000000").encode(
    alt.X("mean(gdp)").scale(domain = (15000, 32000)),
    alt.Y("mean(health)").scale(domain = (1000, 4100)),
    detail = "country"
)
out_pontos + out_pontos.mark_text(
    dx = 3, 
    dy = 3, 
    align = "left"
).transform_aggregate(
    media_health = "mean(health)",
    media_gdp = "mean(gdp)",
    groupby = ("country",)
).transform_filter(
    "datum.media_health < 1500 || datum.media_gdp > 25000 || datum.country == 'Belgium'"
).encode(
    alt.X("media_gdp:Q").scale(domain = (15000, 32000)),
    alt.Y("media_health:Q").scale(domain = (1000, 4100)),
    text = "country"
)

alt.LayerChart(...)

O último tópico importante é sobre anotações nos gráficos: elas são uma peça essencial para "contar uma história". A ideia de anotar um pequeno comentário de texto no gráfico se assemelha a ideia do ggplot: um texto que não está no banco de dados original é inserido como se pertencesse a ele. Uma observação é que é necessário usar o método `transform_calculate` para poder fazer o texto mostrar as quebras de linha, se for preciso ter elas:

In [37]:
base = alt.Chart(organdata).mark_circle().encode(
    x = "roads",
    y = "donors"
)
base

alt.Chart(...)

In [38]:
anotacao = pd.DataFrame({
    "texto": ["Uma surpreendente taxa \nde recuperação"],
    "x": "125",
    "y": "33"
})
anotado = alt.Chart(anotacao).mark_text(align = "right").transform_calculate(
    texto = "split(datum.texto, '\\n')"
).encode(
    x = "x:Q",
    y = "y:Q",
    text = "texto:N"
)
base + anotado

alt.LayerChart(...)

Desenhar retângulos é outra operação semelhante, mas a diferença é que será usada a função `mark_rect`. Os retângulos são especificados por variáveis `x1`, `x2`, `y1` e `y2`, que determinam o espaço coberto por eles. Como toda função `mark`, é possível mudar atributos do retângulo por meio de parâmetros:

In [39]:
retangulo = pd.DataFrame({
    "x1": [125], 
    "x2": [155],
    "y1": [30],
    "y2": [35]
})
marcado = alt.Chart(retangulo).mark_rect(color = "red", opacity = 0.3).encode(
    x = "x1:Q",
    y = "y1:Q",
    x2 = "x2:Q",
    y2 = "y2:Q"
)
base + anotado + marcado

alt.LayerChart(...)